In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy
from scipy.stats import chi2
from scipy.stats import chi2_contingency
from scipy.stats import pearsonr, spearmanr
import pandas.core.algorithms as algos
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec 
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string
max_bin = 20
force_bin = 3
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.svm import SVC
from sklearn import tree
from sklearn.tree import export_graphviz
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import  precision_recall_curve, roc_auc_score, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score,auc, roc_curve, plot_confusion_matrix, classification_report
from IPython.display import Image
import pydotplus
import matplotlib.pyplot as plt
%matplotlib inline
color = sns.color_palette()
seed = 42
!pip install autoimpute
!pip install fancyimpute

from sklearn.impute import SimpleImputer
import time
import gc
!pip install xverse

     |████████████████████████████████| 97 kB 2.8 MB/s 
     |████████████████████████████████| 154 kB 7.8 MB/s 
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29899 sha256=ee25d8be2c82c99116613c11bf3189ab19bc1479c0c7c7a35dc61cfa7d301049
  Stored in directory: /root/.cache/pip/wheels/e3/04/06/a1a7d89ef4e631ce6268ea2d8cde04f7290651c1ff1025ce68
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11353 sha256=1b02c27b4efdfaa55d7c6f00578c1fcb953567c87a3e2d279176bc5f0d7c9934
  Stored in directory: /root/.cache/pip/wheels/72/21/a8/a045cacd9838abd5643f6bfa852c0796a99d6b1494760494e0
Successfully built fancyimpute knnimpute


In [4]:
def application_train_test(num_rows = None, nan_as_category = False):
    # Read data and merge
    df = pd.read_csv('/content/drive/MyDrive/application_train.csv', nrows= num_rows)
    test_df = pd.read_csv('/content/drive/MyDrive/application_test.csv', nrows= num_rows)
    print("Train samples: {}, test samples: {}".format(len(df), len(test_df)))
    df = df.append(test_df).reset_index()
    # Optional: Remove 4 applications with XNA CODE_GENDER (train set)
    df = df[df['CODE_GENDER'] != 'XNA']
    
    # Categorical features with Binary encode (0 or 1; two categories)
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
    # Categorical features with One-Hot encode
    df, cat_cols = one_hot_encoder(df, nan_as_category)
    
    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    # Some simple new features (percentages)
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    del test_df
    gc.collect()
    return df

def bureau_and_balance(num_rows = None, nan_as_category = True):
    bureau = pd.read_csv('/content/drive/MyDrive/bureau.csv', nrows = num_rows)
    bb = pd.read_csv('/content/drive/MyDrive/bureau_balance.csv', nrows = num_rows)
    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)
    
    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
    del bb, bb_agg
    gc.collect()

    # Bureau and bureau_balance numeric features
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    del active, active_agg
    gc.collect()
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    del closed, closed_agg, bureau
    gc.collect()
    return bureau_agg  

def previous_applications(num_rows = None, nan_as_category = True):
    prev = pd.read_csv('/content/drive/MyDrive/previous_application.csv', nrows = num_rows)
    prev, cat_cols = one_hot_encoder(prev, nan_as_category= True)
    # Days 365.243 values -> nan
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    # Add feature: value ask / value received percentage
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    # Previous applications numeric features
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }

    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']
    
    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    # Previous Applications: Approved Applications - only numerical features
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
    # Previous Applications: Refused Applications - only numerical features
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    del refused, refused_agg, approved, approved_agg, prev
    gc.collect()
    return prev_agg

def pos_cash(num_rows = None, nan_as_category = True):
    pos = pd.read_csv('/content/drive/MyDrive/POS_CASH_balance.csv', nrows = num_rows)
    pos, cat_cols = one_hot_encoder(pos, nan_as_category= True)
    # Features
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    
    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    # Count pos cash accounts
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    del pos
    gc.collect()
    return pos_agg

def installments_payments(num_rows = None, nan_as_category = True):
    ins = pd.read_csv('/content/drive/MyDrive/installments_payments.csv', nrows = num_rows)
    ins, cat_cols = one_hot_encoder(ins, nan_as_category= True)
    # Percentage and difference paid in each installment (amount paid and installment value)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    # Days past due and days before due (no negative values)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
    # Features: Perform aggregations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }

    for cat in cat_cols:
        aggregations[cat] = ['mean']
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    # Count installments accounts
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
    del ins
    gc.collect()
    return ins_agg

def credit_card_balance(num_rows = None, nan_as_category = True):
    cc = pd.read_csv('/content/drive/MyDrive/credit_card_balance.csv', nrows = num_rows)
    cc, cat_cols = one_hot_encoder(cc, nan_as_category= True)
    # General aggregations
    cc.drop(['SK_ID_PREV'], axis= 1, inplace = True)
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
    # Count credit card lines
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
    del cc
    gc.collect()
    return cc_agg


In [5]:
#resampling
# umbalanced data
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
def fill_missing(df): 
  column = [col for col in df.columns]
  df[column] = df[column].fillna(df[column].mode().iloc[0])
  return df

def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns
def over_sampling(X, y):
  ros = RandomOverSampler(random_state=25)
  X_ros, y_ros = ros.fit_resample(X, y)
  return X_ros, y_ros 
def under_sampling(X, y):
  rus = RandomUnderSampler(random_state = 42, replacement = True)
  X_rus, y_rus = rus.fit_resample(X, y)
  return X_rus, y_rus
def smothing(X, y):
  smote = SMOTE()
  X_smote, y_smote = smote.fit_resample(X, y)
  return X_smote, y_smote

#Modeling
#Chia data
from sklearn.model_selection import train_test_split
def split_data(X,y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_val, y_train, y_val 

#Scaling data (Tìm hiểu xem có những cách scaling nào)
from sklearn.preprocessing import StandardScaler
def scale(X_train, X_val):
    sc_X = StandardScaler()
    X_train = sc_X.fit_transform(X_train)
    X_val = sc_X.transform(X_val)
    return X_train, X_val

from sklearn.preprocessing import MinMaxScaler
def scale_minmax(X_train, X_val):
  scaler_minmax = MinMaxScaler()
  X_train = scaler_minmax.fit_transform(X_train)
  X_val = scaler_minmax.transform(X_val)
  return X_train, X_val


# Model
def metrics(model, X_val, y_val):
  y_pred = model.predict(X_val)
  ypred_prob = model.predict_proba(X_val)[:,1]
  target_names = ["Class 0", "Class 1"]
  print("AUC",roc_auc_score(y_val, ypred_prob))
  print(classification_report(y_val, y_pred, target_names=target_names))
  return None
  
#print("Classification Accuracy:", round(accuracy_score(y_val, y_pred)*100,2)) 
#print("Precision_score is", round(precision_score(y_val, y_pred)*100,2))
#print("Recall score", round(recall_score(y_val,y_pred)*100,2))
#print("F1 score: ", round(f1_score(y_val,y_pred, average = "weighted") * 100,2))

## Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

def log(X_train,y_train,X_val, y_val):
  log = LogisticRegression()
  model = log.fit(X_train, y_train)
  y_predict = log.predict(X_val)
  a = metrics(model,X_val,y_val)
  return model

 


## Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

def model_random_forest(X_train, y_train, X_val, y_val): 
  clf = RandomForestClassifier(criterion="gini",\
                               random_state = 25,\
                               min_samples_leaf=5)
  model = clf.fit(X_train, y_train)
  y_predict = clf.predict(X_val)
  a = metrics(model,X_val,y_val)
  return model

def rf(X_train,y_train,X_val):
  n_estimators = [5,20,50,100] 
  max_features = ['auto', 'sqrt'] # number of features in consideration at every split
  max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] # maximum number of levels allowed in each decision tree
  min_samples_split = [2, 6, 10] # minimum sample number to split a node
  min_samples_leaf = [1, 3, 4] # minimum sample number that can be stored in a leaf node
  bootstrap = [True, False] # method used to sample data points

  random_grid = {'n_estimators': n_estimators,'max_features': max_features,'max_depth': max_depth,'min_samples_split': min_samples_split,'min_samples_leaf': min_samples_leaf,'bootstrap': bootstrap}
  from sklearn.ensemble import RandomForestRegressor
  rf = RandomForestRegressor()
  from sklearn.model_selection import RandomizedSearchCV
  rf_random = RandomizedSearchCV(estimator = rf,param_distributions = random_grid,
               n_iter = 100, cv = 5, verbose=2, random_state=35, n_jobs = -1)
  rf_random.fit(X_train, y_train)
  print('Best Parameters: ', rf_random.best_params_, ' \n')
  

  #randmf = RandomForestRegressor(n_estimators = 100, min_samples_split = 6, min_samples_leaf= 4, max_features = 'sqrt', max_depth= 120, bootstrap=False) 
  #randmf.fit( X_train, y_train)
  #y_pred = randmf.predict(X_val)
  #ypred_prob = randmf.predict_proba(X_val)[:, 1]
  #print(' AUC : %.6f' % ( roc_auc_score(y_val, ypred_prob)))
  #plot_AUC( y_val, ypred_prob)
  return None


## XGB
def xgb(X_train, y_train, X_val, y_val):
  import xgboost as xgb
  param_init = {
    "objective": "binary:logistic", 
    "booster": "gbtree", 
    "max_depth": 4, # default: 3 only for depthwise
    "n_estimators": 1000, # default: 500  
    "learning_rate": 0.025, # default: 0.05 
    "subsample": 0.7, 
    "colsample_bytree": 0.6,  # default:  1.0
    "colsample_bylevel": 0.5, # default: 1.0
    "random_state": 0,
    
    #
    "silent": True, 
    "n_jobs": 16, 
    
    #
    "tree_method": "hist", # default: auto
    "grow_policy": "lossguide" # default depthwise
    }
    
  param_fit = {
    "eval_metric": "auc", 
    "early_stopping_rounds": 500, # default: 100
    "verbose": 200,
    "eval_set": [(X_train, y_train), (X_val, y_val)]}

  xgb_model = xgb.XGBClassifier(**param_init)
  xgb_model.fit(X_train, y_train, **param_fit)
  evals_result = xgb_model.evals_result()


#lightgbm
def model_LGBM(X_train, y_train, X_val, y_val): 
  clf = lgb.LGBMClassifier(boosting_type = 'goss',
            nthread=4,
            n_estimators=10000,
            learning_rate=0.005134,
            num_leaves=54,
            colsample_bytree=0.508716,
            subsample=1,
            max_depth=10,
            reg_alpha=0.436193,
            reg_lambda=0.479169,
            min_split_gain=0.024766,
            min_child_weight=40,
            silent=-1,
            verbose=-1,
            is_unbalance=False)
  model = clf.fit(X_train, y_train)
  a = metrics(model,X_val,y_val)
  return model




### Chia K-fold - tuning Catboots
def kfold_lightgbm(df, num_folds, stratified = False, debug= False):
    # Divide in training/validation and test data
    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
    del df
    gc.collect()
    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=1001)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=1001)
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

        # LightGBM parameters found by Bayesian optimization
        clf = LGBMClassifier(
            nthread=4,
            n_estimators=10000,
            learning_rate=0.02,
            num_leaves=34,
            colsample_bytree=0.9497036,
            subsample=0.8715623,
            max_depth=8,
            reg_alpha=0.041545473,
            reg_lambda=0.0735294,
            min_split_gain=0.0222415,
            min_child_weight=39.3259775,
            silent=-1,
            verbose=-1, )

        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
            eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)

        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()

    print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    # Write submission file and plot feature importance
    if not debug:
        test_df['TARGET'] = sub_preds
        test_df[['SK_ID_CURR', 'TARGET']].to_csv(submission_file_name, index= False)
    display_importances(feature_importance_df)
    return feature_importance_df

# Display/plot feature importance
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances01.png')

#Metrics Performance
## Đánh giá mô hình thông qua các metrics.
import numpy as np
from sklearn.metrics import  precision_recall_curve, roc_auc_score, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score,auc, roc_curve, plot_confusion_matrix


##AUC - ROC
def plot_AUC( y_val, ypred_prob):
    from sklearn import metrics
    fig, (ax, ax1) = plt.subplots(nrows = 1, ncols = 2, figsize = (15,5))
    
    fpr, tpr, threshold = metrics.roc_curve(y_val, ypred_prob)
    roc_auc = metrics.auc(fpr, tpr)

    ax.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    ax.plot([0, 1], [0, 1],'r--')
    ax.set_title('Receiver Operating Characteristic ',fontsize=10)
    ax.set_ylabel('True Positive Rate',fontsize=20)
    ax.set_xlabel('False Positive Rate',fontsize=15)
    ax.legend(loc = 'lower right', prop={'size': 16})
    plt.subplots_adjust(wspace=1)

In [6]:
df = application_train_test(num_rows = None, nan_as_category = False)
bureau = bureau_and_balance(num_rows = None, nan_as_category = True)
prev = previous_applications(num_rows = None, nan_as_category = True)
pos = pos_cash(num_rows = None, nan_as_category = True)
ins = installments_payments(num_rows = None, nan_as_category = True)
cc = credit_card_balance(num_rows = None, nan_as_category = True)

Train samples: 307511, test samples: 48744


In [7]:
df = df.join(bureau, how='left', on='SK_ID_CURR')
df = df.join(prev, how='left', on='SK_ID_CURR')
df = df.join(pos, how='left', on='SK_ID_CURR')
df = df.join(ins, how='left', on='SK_ID_CURR')
df = df.join(cc, how='left', on='SK_ID_CURR')

In [8]:
df.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df = df.drop('index', axis = 1)

In [10]:
from google.colab import files

df.to_csv('essay.csv', encoding = 'utf-8-sig') 
files.download('essay.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df = fill_missing(df)
df = df.replace((np.inf, -np.inf), 0)

In [ ]:
X = df.drop('TARGET', axis = 1)
y = df.TARGET

In [ ]:
from xverse.transformer import WOE
clf = WOE()
clf.fit(X, y)
clf.woe_df
clf.iv_df

ValueError: ignored

In [ ]:
iv = clf.iv_df
iv_1 = iv[iv['Information_Value']>0.02]
iv_1['Variable_Name']

171                      EXT_SOURCE_2
172                      EXT_SOURCE_3
63              BURO_DAYS_CREDIT_MEAN
170                      EXT_SOURCE_1
65       BURO_DAYS_CREDIT_UPDATE_MEAN
                    ...              
212         INSTAL_AMT_INSTALMENT_MAX
522        REGION_POPULATION_RELATIVE
156    CLOSED_DAYS_CREDIT_ENDDATE_MAX
216     INSTAL_DAYS_ENTRY_PAYMENT_SUM
341                      PAYMENT_RATE
Name: Variable_Name, Length: 67, dtype: object

In [ ]:
iv_1

,Variable_Name,Information_Value
171,EXT_SOURCE_2,0.247521
172,EXT_SOURCE_3,0.171841
63,BURO_DAYS_CREDIT_MEAN,0.105685
170,EXT_SOURCE_1,0.085756
65,BURO_DAYS_CREDIT_UPDATE_MEAN,0.084407
...,...,...
212,INSTAL_AMT_INSTALMENT_MAX,0.020582
522,REGION_POPULATION_RELATIVE,0.020442
156,CLOSED_DAYS_CREDIT_ENDDATE_MAX,0.020319
216,INSTAL_DAYS_ENTRY_PAYMENT_SUM,0.020199


In [ ]:
X=df[[col for col in iv_1['Variable_Name']]]
X = clf.transform(X)
y = df.TARGET

In [ ]:
X_train, X_val, y_train, y_val = split_data(X,y)
#Sampling
X_ros, y_ros = over_sampling(X_train, y_train)
X_rus, y_rus = under_sampling(X_train, y_train)
X_smooth, y_smooth = smothing(X_train, y_train)

In [ ]:
log(X_train, y_train, X_val, y_val)

AUC 0.7409154239114759
              precision    recall  f1-score   support

     Class 0       0.92      1.00      0.96     50928
     Class 1       0.53      0.00      0.01      4429

    accuracy                           0.92     55357
   macro avg       0.72      0.50      0.48     55357
weighted avg       0.89      0.92      0.88     55357



LogisticRegression()

In [ ]:
print("Oversampling:")
model1 = log(X_ros, y_ros, X_val, y_val)
print("\nUndersampling:")
model2 = log(X_rus, y_rus, X_val, y_val)
print("\nSmote: ")
model3 = log(X_smooth, y_smooth, X_val, y_val)

Oversampling:


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


AUC 0.741480971600156
              precision    recall  f1-score   support

     Class 0       0.96      0.68      0.79     50928
     Class 1       0.16      0.68      0.25      4429

    accuracy                           0.68     55357
   macro avg       0.56      0.68      0.52     55357
weighted avg       0.90      0.68      0.75     55357


Undersampling:
AUC 0.7411119790541689
              precision    recall  f1-score   support

     Class 0       0.96      0.68      0.79     50928
     Class 1       0.15      0.68      0.25      4429

    accuracy                           0.68     55357
   macro avg       0.56      0.68      0.52     55357
weighted avg       0.90      0.68      0.75     55357


Smote: 


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


AUC 0.7351036028923411
              precision    recall  f1-score   support

     Class 0       0.96      0.69      0.80     50928
     Class 1       0.16      0.66      0.25      4429

    accuracy                           0.68     55357
   macro avg       0.56      0.67      0.53     55357
weighted avg       0.89      0.68      0.76     55357



In [ ]:
#Hyperparameters tuning with randomsearchcv
def randomsearchcv_lgbm(X_train, y_train, X_val, y_val):
  clf = lgb.LGBMClassifier()
  param_dist = { "learning_rate": np.linspace(0,0.2,5),
               "max_depth": randint(3, 10),
                "min_split_gain": np.linspace(1, 10, 1),
                "num_iterations": randint(100, 10000),
                "min_data_in_leaf": randint(3, 10),
                "min_gain_to_split": randint(1, 10),
                "max_bin": randint(10, 100)}
               
  rscv = RandomizedSearchCV(clf , param_dist, scoring='accuracy', cv =5)
  rscv.fit(X_train,y_train)
  best_params = rscv.best_estimator_
  y_predict = best_params.predict(X_val)
  y_predict_proba = best_params.predict_proba(X_val)[:, 1]
  a = metrics(rscv,X_val,y_val)
  return rscv

!pip3 install catboost
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
def randomsearchcv_catboost(X_train, y_train, X_val, y_val):
  clf = CatBoostClassifier()
  param_dist = { "learning_rate": np.linspace(0,0.2,5),
               "max_depth": randint(3, 10),
                "iterations": randint(10, 100)}
               
  rscv = RandomizedSearchCV(clf , param_dist, scoring='accuracy', cv =5)
  rscv.fit(X_train,y_train)
  best_params = rscv.best_estimator_
  y_predict = best_params.predict(X_val)
  y_predict_proba = best_params.predict_proba(X_val)[:, 1]
  a = metrics(rscv,X_val,y_val)
  return rscv

In [ ]:
y = df.TARGET
X = df.drop('TARGET', axis = 1)
X_train, X_val, y_train, y_val = split_data(X,y)
X_train, X_val = scale(X_train, X_val)
#Sampling
X_ros, y_ros = over_sampling(X_train, y_train)
X_rus, y_rus = under_sampling(X_train, y_train)
X_smooth, y_smooth = smothing(X_train, y_train)

In [ ]:
rscv_catboost_ros = randomsearchcv_catboost(X_ros, y_ros, X_val, y_val)

In [ ]:
rscv_catboost_rus = randomsearchcv_catboost(X_rus, y_rus, X_val, y_val)

In [ ]:
rscv_catboost_smooth = randomsearchcv_catboost(X_smooth, y_smooth, X_val, y_val)

In [ ]:
rscv_lgbm_ros = randomsearchcv_lgbm(X_ros, y_ros, X_val, y_val)

In [ ]:
rscv_lgbm_rus = randomsearchcv_lgbm(X_rus, y_rus, X_val, y_val)

In [ ]:
rscv_lgbm_smooth = randomsearchcv_lgbm(X_smooth, y_smooth, X_val, y_val)

In [ ]:
feat_importance = kfold_lightgbm(df, num_folds= 10, stratified= False)